In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch  as th
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [47]:
class SelfSupervisedProjection:
    def __init__(self, input_dim, projection_dim, random_state=None):
        self.random_state = random_state
        self.input_dim = input_dim
        self.projection_dim = projection_dim



    def project(self, X):
        th.manual_seed(self.random_state)
        self.projection_matrix = th.randn(self.projection_dim, self.input_dim) / th.sqrt(th.tensor(self.input_dim, dtype=th.float))
        projected_matrix = th.matmul(X, self.projection_matrix.T)
        return projected_matrix




    def fit(self, X):
        projected_matrix = transform(X)
        return projected_matrix


    def predict(self, X):

        return None


In [20]:
filepath = 'data/cassette-th-data.pck'
xtrain, xvalid, ytrain, yvalid = np.load(filepath, allow_pickle = True)
input_dim = xtrain[0].shape
input_dim[0]

600

In [65]:
test = SelfSupervisedProjection(input_dim=input_dim[0], projection_dim = 100, random_state= 42)
a = test.project(xtrain)
a

tensor([[  2.5422,  -6.7154,  13.2885,  ...,  -6.2785,  15.3018,   7.4020],
        [  3.7200,  38.1613, -29.8826,  ...,  10.5134, -14.9686,  10.4928],
        [ 14.5077, -15.3107,   7.1610,  ...,  18.9185, -24.2584,  -6.4876],
        ...,
        [199.6864, -60.1919, 146.5060,  ..., -44.3121,  54.3268,   6.6187],
        [  8.7187, -11.0354,  -7.9052,  ...,   4.4007,   2.9942,   8.2963],
        [  9.3255,   5.2186, -21.3133,  ...,   7.6667,  11.4504,  30.0089]])